In [1]:
from constant import HEADER_NAME, COLUMNS_TO_DROP, REPORTDAYS, FIRST_ROW_AFTER_BURNIN
import pandas as pd
from plot_helper import coloring_legend, df_col_replace
import copy
import numpy as np

In [2]:
def t01_and_1_reporter(dflist_arg, drug, mdr_tag):
  global REPORTDAYS, FIRST_ROW_AFTER_BURNIN
  def calculate_for_one(data):
    idx_one_percent = -1
    idx_ten_percent = -1
    for idx,val in enumerate(data):
      if val > 0.01:
        idx_one_percent = idx
        break
    for idx,val in enumerate(data):
      if val > 0.1:
        idx_ten_percent = idx
        break
    return idx_one_percent, idx_ten_percent
  
  t_onep = []
  t_tenp = []
  POSTBURNIN_REPORTDAYS = REPORTDAYS[FIRST_ROW_AFTER_BURNIN:]
  dflist = copy.deepcopy(dflist_arg)
  for i in range(len(dflist)):
    dflist[i] = df_col_replace(dflist[i], drug, option=1)
    dflist[i] = dflist[i].iloc[FIRST_ROW_AFTER_BURNIN:]
    dflist[i].reset_index(drop=True, inplace=True)
    idx_one_percent, idx_ten_percent = calculate_for_one(dflist[i][mdr_tag])
    if idx_one_percent == -1: t_onep.append(float('inf'))
    else: t_onep.append(POSTBURNIN_REPORTDAYS[idx_one_percent])
    if idx_ten_percent == -1: t_tenp.append(float('inf'))
    else: t_tenp.append(POSTBURNIN_REPORTDAYS[idx_ten_percent])
  
  # IQR, convert to year after burn-in
  print(np.quantile(t_onep, [0.025, 0.25, 0.5, 0.75, 0.99975]) / 365 - 10)
  print(np.quantile(t_tenp, [0.025, 0.25, 0.5, 0.75, 0.99975]) / 365 - 10)

In [3]:
# path to 100 simulation outputs
file_path_adpcyc = 'raw_data/set7_ac/monthly/set7ac_%smonthly_data_0.txt'
# path to ac outputs, with alternative initial genotypes
file_path_aac = 'raw_data/alt_init_geno/set7_aac/monthly/set7aac_%smonthly_data_0.txt'

In [4]:
dflist = []
for i in range(1,101):
  dflist.append(pd.read_csv(file_path_adpcyc % i, index_col=False, \
                            names=HEADER_NAME, sep='\t').drop(columns=COLUMNS_TO_DROP))

In [5]:
dflist_aac = []
for i in range(1,101):
  dflist_aac.append(pd.read_csv(file_path_aac % i, index_col=False, \
                            names=HEADER_NAME, sep='\t').drop(columns=COLUMNS_TO_DROP))

In [6]:
# row 1, '2-2' circles
t01_and_1_reporter(dflist, 'DHA-PPQ', '2-2')

[ 4.96356164  6.26027397  7.1369863   7.84657534 12.15543425]
[ 8.17541096 11.76506849 14.55753425 16.55753425         inf]


In [7]:
# row 2, '2-4' circles
t01_and_1_reporter(dflist, 'ASAQ', '2-4')

[ 5.16287671  7.32191781  8.26027397  9.11780822 12.74173562]
[ 9.53876712 13.17808219 14.85068493 16.36917808 19.9245774 ]


In [8]:
# row 3, '2-2' triangles
t01_and_1_reporter(dflist, 'AL', '2-2')

[ 5.50684932  6.76164384  7.55205479  8.42739726 16.72175959]
[12.38554795 15.81027397 18.85479452         inf         inf]


In [9]:
# row 4, '2-2' triangles
t01_and_1_reporter(dflist_aac, 'AL', '2-2')

[ 5.54438356  7.34246575  8.21780822  9.42739726 16.42020479]
[ 9.76164384 13.40890411 15.51506849 17.76712329         inf]
